# 01. Learning LangGraph - Agent Executor

modified from https://github.com/langchain-ai/langgraph/blob/main/examples/agent_executor/base.ipynb

In [2]:
import os
from dotenv import load_dotenv

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv('LANGSMITH_API_KEY')
os.environ["LANGCHAIN_PROJECT"] = "LangGraph_01"

## Making the GraphState

In [3]:
from typing import TypedDict, Annotated, List, Union
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
import operator


class AgentState(TypedDict):
   # The input string
   input: str
   # The list of previous messages in the conversation
   chat_history: list[BaseMessage]
   # The outcome of a given call to the agent
   # Needs `None` as a valid type, since this is what this will start as
   agent_outcome: Union[AgentAction, AgentFinish, None]
   # List of actions and corresponding observations
   # Here we annotate this with `operator.add` to indicate that operations to
   # this state should be ADDED to the existing values (not overwrite it)
   intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]

## Custom Tools

**Tools**  

Tools are interfaces that an agent can use to interact with the world. They combine a few things:



1.   The name of the tool
2.   A description of what the tool is
3.   JSON schema of what the inputs to the tool are
4.   The function to call


Whether the result of a tool should be returned directly to the user

In [5]:
from langchain.tools import BaseTool, StructuredTool, Tool, tool

In [6]:
import random

@tool("lower_case", return_direct=True)
def to_lower_case(input:str) -> str:
  """Returns the input as all lower case."""
  return input.lower()

@tool("random_number", return_direct=True)
def random_number_maker(input:str) -> str:
    """Returns a random number between 0-100."""
    return random.randint(0, 100)

tools = [to_lower_case,random_number_maker]

In [7]:
random_number_maker.run('random')

39

Failed to multipart ingest runs: Connection error caused failure to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. Please confirm your internet connection. ConnectTimeout(MaxRetryError("HTTPSConnectionPool(host='api.smith.langchain.com', port=443): Max retries exceeded with url: /runs/multipart (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000017933344F80>, 'Connection to api.smith.langchain.com timed out. (connect timeout=10.0)'))"))
Content-Length: 61913
API Key: lsv2_********************************************cetrace=c295db05-00b7-42f1-94d9-82508e932889,id=9efd3a66-5197-407f-ad9a-e99a716794d9; trace=c295db05-00b7-42f1-94d9-82508e932889,id=32d0f848-7650-4c76-a4f3-7605accfb9c7; trace=c295db05-00b7-42f1-94d9-82508e932889,id=91740052-b31c-4c87-abbb-83978cf6f23d; trace=c295db05-00b7-42f1-94d9-82508e932889,id=6e2f72a3-2048-46b1-8421-b98de503e9a4; trace=c295db05-00b7-42f1-94d9-82508e932889,id=2b58a5ac-de8b-45c5-811d-d4b19493ed9a; tr

In [8]:
to_lower_case.run('SAM')

'sam'

## Agent - with new create_open_ai

In [19]:
from langchain import hub
from langchain.agents import create_structured_chat_agent
from tools.initialize_cerebras import init_cerebras

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/structured-chat-agent")

# Choose the LLM that will drive the agent
client, llm = init_cerebras()

# Construct the OpenAI Functions agent
agent_runnable = create_structured_chat_agent(llm,
                                               tools,
                                               prompt)

In [20]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.System

In [21]:
prompt.get_prompts()

[ChatPromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.Syste

In [22]:
inputs = {"input": "give me a random number and then write in words and make it lower case.",
          "chat_history": [],
          "intermediate_steps":[]}

agent_outcome = agent_runnable.invoke(inputs)

In [23]:
agent_outcome

AgentAction(tool='random_number', tool_input='generate a number', log='Thought: I need to generate a random number first.\nAction:\n```\n{\n  "action": "random_number",\n  "action_input": "generate a number"\n}\n```')

In [24]:
type(agent_runnable)

langchain_core.runnables.base.RunnableSequence

## Nodes

In [25]:
from langchain_core.agents import AgentFinish
from langgraph.prebuilt.tool_executor import ToolExecutor

tool_executor = ToolExecutor(tools)

C:\Users\ncvn\AppData\Local\Temp\ipykernel_3756\2882498864.py:4: LangGraphDeprecationWarning: ToolExecutor is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  tool_executor = ToolExecutor(tools)


In [26]:
# Define the agent/graph
def run_agent(data):
    agent_outcome = agent_runnable.invoke(data)
    return {"agent_outcome": agent_outcome}

# Define the function to execute tools
def execute_tools(data):
    # Get the most recent agent_outcome - this is the key added in the `agent` above
    agent_action = data['agent_outcome']
    # Execute the tool
    output = tool_executor.invoke(agent_action)
    print(f"The agent action is {agent_action}")
    print(f"The tool result is: {output}")
    # Return the output
    return {"intermediate_steps": [(agent_action, str(output))]}

# Define logic that will be used to determine which conditional edge to go down
def should_continue(data):
    # If the agent outcome is an AgentFinish, then we return `exit` string
    # This will be used when setting up the graph to define the flow
    if isinstance(data['agent_outcome'], AgentFinish):
        return "end"
    # Otherwise, an AgentAction is returned
    # Here we return `continue` string
    # This will be used when setting up the graph to define the flow
    else:
        return "continue"

## Define the graph

In [27]:
from langgraph.graph import END, StateGraph

# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", run_agent)
workflow.add_node("action", execute_tools)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "action",
        # Otherwise we finish.
        "end": END
    }
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge('action', 'agent')

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()

In [29]:
workflow.branches

defaultdict(dict,
            {'agent': {'should_continue': Branch(path=should_continue(tags=None, recurse=True, func_accepts_config=False, func_accepts={'writer': False, 'store': False}), ends={'continue': 'action', 'end': '__end__'}, then=None)}})

In [28]:
workflow.nodes, workflow.edges

({'agent': StateNodeSpec(runnable=agent(tags=None, recurse=True, func_accepts_config=False, func_accepts={'writer': False, 'store': False}), metadata=None, input=<class '__main__.AgentState'>, retry_policy=None, ends=()),
  'action': StateNodeSpec(runnable=action(tags=None, recurse=True, func_accepts_config=False, func_accepts={'writer': False, 'store': False}), metadata=None, input=<class '__main__.AgentState'>, retry_policy=None, ends=())},
 {('__start__', 'agent'), ('action', 'agent')})

In [31]:
workflow.channels #['intermediate_steps']

{'input': <langgraph.channels.last_value.LastValue at 0x179330a30c0>,
 'chat_history': <langgraph.channels.last_value.LastValue at 0x179330a3080>,
 'agent_outcome': <langgraph.channels.last_value.LastValue at 0x17932dcd680>,
 'intermediate_steps': <langgraph.channels.binop.BinaryOperatorAggregate at 0x17932fe0f80>}

In [32]:
inputs = {"input": "give me a random number and then write in words and make it lower case.", "chat_history": []}
for s in app.stream(inputs):
    print(list(s.values())[0])
    print("----")

{'agent_outcome': AgentAction(tool='random_number', tool_input='Give me a random number', log='Thought: To give you a random number, I should use the random_number tool. \n\nAction:\n```\n{\n  "action": "random_number",\n  "action_input": "Give me a random number"\n}\n```')}
----
The agent action is tool='random_number' tool_input='Give me a random number' log='Thought: To give you a random number, I should use the random_number tool. \n\nAction:\n```\n{\n  "action": "random_number",\n  "action_input": "Give me a random number"\n}\n```'
The tool result is: 49
{'intermediate_steps': [(AgentAction(tool='random_number', tool_input='Give me a random number', log='Thought: To give you a random number, I should use the random_number tool. \n\nAction:\n```\n{\n  "action": "random_number",\n  "action_input": "Give me a random number"\n}\n```'), '49')]}
----
{'agent_outcome': AgentAction(tool='lower_case', tool_input='Forty-nine', log='To write the number in words and make it lower case, I will

In [36]:
inputs = {"input": "give me a random number and then write in words and make it lower case", "chat_history": []}

output = app.invoke(inputs)

The agent action is tool='random_number' tool_input='random' log='Action:\n```\n{\n  "action": "random_number",\n  "action_input": "random"\n}\n```'
The tool result is: 62
The agent action is tool='lower_case' tool_input='sixty-two' log='To write the number in words and make it lower case, I can first write the number in words and then convert it to lower case.\n\nAction:\n```\n{\n  "action": "lower_case",\n  "action_input": "sixty-two"\n}\n```'
The tool result is: sixty-two


In [38]:
output.get("agent_outcome").return_values['output']

'i have a random number, wrote it in words, and made it lower case. the number is sixty-two.'

In [39]:
output.get("intermediate_steps")

[(AgentAction(tool='random_number', tool_input='random', log='Action:\n```\n{\n  "action": "random_number",\n  "action_input": "random"\n}\n```'),
  '62'),
 (AgentAction(tool='lower_case', tool_input='sixty-two', log='To write the number in words and make it lower case, I can first write the number in words and then convert it to lower case.\n\nAction:\n```\n{\n  "action": "lower_case",\n  "action_input": "sixty-two"\n}\n```'),
  'sixty-two')]

In [40]:
inputs = {"input": "does it get cold in SF in Jan?", "chat_history": []}

output = app.invoke(inputs)

The agent action is tool='lower_case' tool_input='Is it cold in San Francisco in January?' log='Thought: To determine if it gets cold in San Francisco in January, we should consider the typical winter weather in the area. However, the question doesn\'t explicitly ask for a direct answer, so I\'ll provide one by rephrasing it.\n\nAction:\n```\n{\n  "action": "lower_case",\n  "action_input": "Is it cold in San Francisco in January?"\n}\n```'
The tool result is: is it cold in san francisco in january?


In [41]:
output.get("agent_outcome").return_values['output']


'Yes, it can get quite chilly in San Francisco in January, with average temperatures ranging from 45°F to 57°F (7°C to 14°C).'

In [42]:
output.get("intermediate_steps")

[(AgentAction(tool='lower_case', tool_input='Is it cold in San Francisco in January?', log='Thought: To determine if it gets cold in San Francisco in January, we should consider the typical winter weather in the area. However, the question doesn\'t explicitly ask for a direct answer, so I\'ll provide one by rephrasing it.\n\nAction:\n```\n{\n  "action": "lower_case",\n  "action_input": "Is it cold in San Francisco in January?"\n}\n```'),
  'is it cold in san francisco in january?')]